In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [2]:
data = np.genfromtxt("spambase/spambase.data", delimiter=',')
X = data[:,:57]
Y = data[:,-1]
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [3]:
Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.05, shuffle=True)

In [4]:
print("Number of features:\t", X.shape[1])
print("Number of training data points:\t", Xtr.shape[0])
print("Number of test data points:\t", Xte.shape[0])

Number of features:	 57
Number of training data points:	 4370
Number of test data points:	 231


In [5]:
#pca = PCA(n_components=30)
#Xpca = pca.fit_transform(X)
#print(Xpca)

In [6]:
logreg = LogisticRegression(tol=1e-8, C=10.0, random_state=0, solver='lbfgs', multi_class='multinomial')
clf = logreg.fit(Xtr,Ytr)
#clf  = LogisticRegression(tol=1e-8, C=10.0).fit(Xtr, Ytr)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [7]:
print("training score:\t", clf.score(Xtr,Ytr))
print("test score:\t", clf.score(Xte,Yte))

training score:	 0.9320366132723112
test score:	 0.9437229437229437


In [8]:
Yhat = clf.predict(Xte)

In [9]:
err = 0
for i in range(len(Xte)):
    if Yhat[i] != Yte[i]:
        err += 1
print("Error rate: ", err/len(Xte))

Error rate:  0.05627705627705628
